In [1]:
pip install git+https://github.com/mimoralea/gym-walk#egg=gym-walk


  Cloning https://github.com/mimoralea/gym-walk to /tmp/pip-install-nx3p4tvk/gym-walk_33b1f516695c4731b2d4694aadda7faf
  Running command git clone --filter=blob:none --quiet https://github.com/mimoralea/gym-walk /tmp/pip-install-nx3p4tvk/gym-walk_33b1f516695c4731b2d4694aadda7faf
  Resolved https://github.com/mimoralea/gym-walk to commit 5999016267d6de2f5a63307fb00dfd63de319ac1
  Preparing metadata (setup.py) ... done
  Created wheel for gym-walk: filename=gym_walk-0.0.2-py3-none-any.whl size=4054 sha256=3372a7dd363d5b877d20f873e55302e5a285adc00276a58421fab7c5a2722e9f
  Stored in directory: /tmp/pip-ephem-wheel-cache-_0m_ousa/wheels/24/fe/c4/0cbc7511d29265bad7e28a09311db3f87f0cafba74af54d530
Successfully built gym-walk


In [2]:
import warnings ; warnings.filterwarnings('ignore')

import gym, gym_walk
import numpy as np

import random
import warnings

warnings.filterwarnings('ignore', category=DeprecationWarning)
np.set_printoptions(suppress=True)
random.seed(123); np.random.seed(123)

In [3]:
def print_policy(pi, P, action_symbols=('<', 'v', '>', '^'), n_cols=4, title='Policy:'):
    print(title)
    arrs = {k: v for k, v in enumerate(action_symbols)}
    for s in range(len(P)):
        a = pi[s]  # Access the value associated with state s
        print("| ", end="")
        if np.all([done for action in P[s].values() for _, _, _, done in action]):
            print("".rjust(9), end=" ")
        else:
            print(str(s).zfill(2), arrs[a].rjust(6), end=" ")
        if (s + 1) % n_cols == 0:
            print("|")

In [4]:
def print_state_value_function(V, P, n_cols=4, prec=3, title='State-value function:'):
    print(title)
    for s in range(len(P)):
        v = V[s]
        print("| ", end="")
        if np.all([done for action in P[s].values() for _, _, _, done in action]):
            if title == 'State-value function:':
                print("".rjust(9), end=" ")
            else:
                print("".rjust(24), end=" ")
        else:
            print(str(s).zfill(2), '{}'.format(np.round(v, prec)).rjust(6), end=" ")
        if (s + 1) % n_cols == 0: print("|")

In [5]:
env = gym.make('FrozenLake-v1')
P = env.env.P
init_state = env.reset()

In [6]:
P

{0: {0: [(0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 4, 0.0, False)],
  1: [(0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 4, 0.0, False),
   (0.3333333333333333, 1, 0.0, False)],
  2: [(0.3333333333333333, 4, 0.0, False),
   (0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False)],
  3: [(0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 0, 0.0, False)]},
 1: {0: [(0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 5, 0.0, True)],
  1: [(0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 5, 0.0, True),
   (0.3333333333333333, 2, 0.0, False)],
  2: [(0.3333333333333333, 5, 0.0, True),
   (0.3333333333333333, 2, 0.0, False),
   (0.3333333333333333, 1, 0.0, False)],
  3: [(0.3333333333333333, 2, 0.0, False),
   (0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False)]},
 2:

In [7]:

def decay_schedule(init_val,min_val,decay_ratio,max_steps,log_start=2,log_base=10):
    decay_steps = int(max_steps * decay_ratio)
    rem_steps = max_steps - decay_steps

    values = np.logspace(log_start,0,decay_steps,base=log_base,endpoint=True)[::-1]

    values = (values - values.min()) / (values.max() - values.min())

    values = (init_val - min_val) * values + min_val

    values = np.pad(values,(0,rem_steps),'edge')

    return values

In [8]:
from itertools import count
import numpy as np

def gen_traj(select_action,Q,epsilon,env,max_steps=200):
    done,traj = False,[]

    while not done:
        state = env.reset()

        for t in count():
            action = select_action(state,Q,epsilon)
            next,reward,done,_ = env.step(action)
            exp = (state,action,reward,next,done)
            traj.append(exp)

            if done:
                break
            if t>=max_steps-1:
                traj = []
                break

            state = next
        return np.array(traj, np.object)


In [9]:
import numpy as np
from tqdm import tqdm

def mc_control(env, gamma=1.0, init_alpha=0.5, min_alpha=0.01, alpha_decay_ratio=0.5,
               init_epsilon=1.0, min_epsilon=0.1, epsilon_decay_ratio=0.9,
               n_episodes=3000, max_steps=200, first_visit=True):

    nS, nA = env.observation_space.n, env.action_space.n

    disc = np.logspace(0, max_steps, num=max_steps, base=gamma, endpoint=False)

    def decay_schedule(init_value, min_value, decay_ratio, n):
        return np.maximum(min_value, init_value * (decay_ratio ** np.arange(n)))

    alphas = decay_schedule(init_alpha, min_alpha, alpha_decay_ratio, n_episodes)
    epsilons = decay_schedule(init_epsilon, min_epsilon, epsilon_decay_ratio, n_episodes)

    Q = np.zeros((nS, nA), dtype=np.float64)
    Q_track = np.zeros((n_episodes, nS, nA), dtype=np.float64)

    def select_action(state, Q, epsilon):
        return np.argmax(Q[state]) if np.random.random() > epsilon else np.random.randint(nA)

    for e in tqdm(range(n_episodes), leave=False):
        traj = gen_traj(select_action, Q, epsilons[e], env, max_steps)
        visited = np.zeros((nS, nA), dtype=np.bool)

        for t, (state, action, reward, _, _) in enumerate(traj):
            if visited[state][action] and first_visit:
                continue
            visited[state][action] = True

            n_steps = len(traj[t:])
            G = np.sum(disc[:n_steps] * traj[t:, 2])
            Q[state][action] = Q[state][action] + alphas[e] * (G - Q[state][action])

        Q_track[e] = Q

    V = np.max(Q, axis=1)
    pi = {s: np.argmax(Q[s]) for s in range(nS)}

    return Q, V, pi

In [10]:
optimal_Q, optimal_V, optimal_pi = mc_control (env,n_episodes=150000)


In [11]:
print_state_value_function(optimal_Q, P, n_cols=4, prec=2, title='Action-value function:')
print_state_value_function(optimal_V, P, n_cols=4, prec=2, title='State-value function:')
print_policy(optimal_pi, P)

Action-value function:
| 00 [0.35 0.39 0.36 0.37] | 01 [0.16 0.16 0.13 0.34] | 02 [0.2  0.27 0.23 0.21] | 03 [0.15 0.1  0.08 0.26] |
| 04 [0.4  0.26 0.25 0.32] |                          | 06 [0.21 0.17 0.28 0.11] |                          |
| 08 [0.28 0.35 0.33 0.43] | 09 [0.38 0.52 0.35 0.31] | 10 [0.53 0.43 0.33 0.26] |                          |
|                          | 13 [0.42 0.46 0.65 0.38] | 14 [0.64 0.83 0.74 0.72] |                          |
State-value function:
| 00   0.39 | 01   0.34 | 02   0.27 | 03   0.26 |
| 04    0.4 |           | 06   0.28 |           |
| 08   0.43 | 09   0.52 | 10   0.53 |           |
|           | 13   0.65 | 14   0.83 |           |
Policy:
| 00      v | 01      ^ | 02      v | 03      ^ |
| 04      < |           | 06      > |           |
| 08      ^ | 09      v | 10      < |           |
|           | 13      > | 14      v |           |


In [12]:
def probability_success(env, pi, goal_state, n_episodes=100, max_steps=200):
    random.seed(123)
    np.random.seed(123)
    env.seed(123)
    results = []

    for _ in range(n_episodes):
        state, done, steps = env.reset(), False, 0
        while not done and steps < max_steps:
            action = pi[state]  # Access the action for the current state from the policy dictionary
            state, _, done, _ = env.step(action)
            steps += 1
        results.append(state == goal_state)

    return np.sum(results) / len(results)

In [13]:
def mean_return(env, pi, n_episodes=100, max_steps=200):
    random.seed(123)
    np.random.seed(123)
    env.seed(123)
    results = []

    for _ in range(n_episodes):
        state, done, steps = env.reset(), False, 0
        results.append(0.0)
        while not done and steps < max_steps:
            action = pi[state]  # Access the action for the current state from the policy dictionary
            state, reward, done, _ = env.step(action)
            results[-1] += reward
            steps += 1
    return np.mean(results)

In [14]:
def results(env,optimal_pi,optimal_V,P):
    print('Reaches goal {:.2f}%. Obtains an average undiscounted return of {:.4f}.'.format(
    probability_success(env, optimal_pi, goal_state=goal_state)*100,
    mean_return(env, optimal_pi)))


In [15]:
goal_state = 15
results(env,optimal_pi,optimal_V,P)

Reaches goal 47.00%. Obtains an average undiscounted return of 0.4700.
